In [46]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import scipy.sparse as sp
from scipy.special import expit
import pickle
import csv
import scipy.sparse as sparse
import copy
import itertools
from lightfm import LightFM
import lightfm.evaluation
import sys
import implicit
plt.style.use('ggplot')

In [47]:
raw_df = pd.read_csv('cat_de_3year.csv', encoding = "utf-8")#, nrows = 100000)

In [92]:
## to start let us take a sample of our data
sample_df = raw_df.sample(10000000)
sample_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID
1146150,203908573,48613563,0,16-JAN-15,1266101,16111,2550159,1.0000,11854,10178
9164653,247098575,58624851,0,03-OCT-15,1641805,81992388,11901040,2.0000,65444,47079
6710425,239585081,56878941,0,22-AUG-15,1598726,84082749,133051,0.3333,66004,47197
15038847,281959795,66622125,0,30-MAR-16,1700807,13857178,3446867,1.0000,35464,25635
39298845,421430351,98809847,0,21-NOV-17,1474715,71076777,12070597,1.0000,63294,45739


In [93]:
# remove non-used columns
#sample_df.drop(['SCM_N_ORDER_POS_NR','SCM_N_ORDER_NR_NC', 'SCM_N_ORDER_NR_NC', 'PPR_N_KEY_PHYS_PRODUCT', 'SCM_N_ORDER_NR', 'ART_N_KEY_ARTICLE','PPR_N_PHY_PRODUCT_ID'], axis=1, inplace=True)
sample_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID
1146150,203908573,48613563,0,16-JAN-15,1266101,16111,2550159,1.0000,11854,10178
9164653,247098575,58624851,0,03-OCT-15,1641805,81992388,11901040,2.0000,65444,47079
6710425,239585081,56878941,0,22-AUG-15,1598726,84082749,133051,0.3333,66004,47197
15038847,281959795,66622125,0,30-MAR-16,1700807,13857178,3446867,1.0000,35464,25635
39298845,421430351,98809847,0,21-NOV-17,1474715,71076777,12070597,1.0000,63294,45739


In [94]:
sample_renamed_df = sample_df.rename(index=str, columns = {'CUS_N_KEY_CUSTOMER': 'uid', 
                                                   'ART_N_KEY_ARTICLE': 'mid', 
                                                   'TDT_T_KEY_ORDER_DATE': 'date', # it looks like this works a bit better
                                                   'SCM_D_QUANTITY': 'eventStrength'})
sample_renamed_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,date,mid,PPR_N_KEY_PHYS_PRODUCT,uid,eventStrength,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID
1146150,203908573,48613563,0,16-JAN-15,1266101,16111,2550159,1.0000,11854,10178
9164653,247098575,58624851,0,03-OCT-15,1641805,81992388,11901040,2.0000,65444,47079
6710425,239585081,56878941,0,22-AUG-15,1598726,84082749,133051,0.3333,66004,47197
15038847,281959795,66622125,0,30-MAR-16,1700807,13857178,3446867,1.0000,35464,25635
39298845,421430351,98809847,0,21-NOV-17,1474715,71076777,12070597,1.0000,63294,45739


In [95]:
n_users = sample_renamed_df.uid.unique().shape[0]
n_items = sample_renamed_df.mid.unique().shape[0]

print('Number of users: {}'.format(n_users))
print('Number of models: {}'.format(n_items))
print('Sparsity: {:4.3f}%'.format(float(sample_renamed_df.shape[0]) / float(n_users*n_items) * 100))

Number of users: 904417
Number of models: 20820
Sparsity: 0.053%


In [96]:
sample_renamed_df.uid[2]

133051

In [85]:
raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == 3334225].head(30)

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID
207993,256259015,60750707,0,22-NOV-15,1511000,72541511,3334225,1.0000,64176,46182
207994,256259013,60750707,0,22-NOV-15,1606802,81297991,3334225,3.0000,65244,47059
2011150,224629683,53406262,0,23-MAY-15,1569004,80851953,3334225,2.0000,65233,47048
2011151,224629684,53406262,0,23-MAY-15,27039,10652,3334225,1.0000,16748,13777
2044265,207652887,49469555,0,07-FEB-15,1453924,26302120,3334225,1.0000,44150,33416
2044320,207652881,49469555,0,07-FEB-15,27039,10652,3334225,1.0000,16748,13777
2044321,207652882,49469555,0,07-FEB-15,293394,14536,3334225,1.0000,19588,15938
2044322,207652883,49469555,0,07-FEB-15,1495511,70549078,3334225,1.0000,63087,45648
2123543,225335285,53568291,0,27-MAY-15,1569004,80851953,3334225,1.0000,65233,47048
2485171,224629685,53406262,0,23-MAY-15,1453924,26302120,3334225,1.0000,44150,33416


In [87]:
raw_df['ART_N_KEY_ARTICLE'].unique().size

21260

In [88]:
raw_df['PPR_N_PHY_ARTICLE_ID'].unique().size

5095

In [89]:
raw_df['PPR_N_PHY_PRODUCT_ID'].unique().size

2288

In [91]:
raw_df[raw_df['ART_N_KEY_ARTICLE'] == 1281816].head(30)

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID
4843,210626987,50166085,0,25-FEB-15,1281816,53757806,10893687,0.2500,57358,43259
4844,210626987,50166085,0,25-FEB-15,1281816,53757818,10893687,0.2500,57353,43259
4845,210626987,50166085,0,25-FEB-15,1281816,53757804,10893687,0.2501,57359,43259
4846,210626987,50166085,0,25-FEB-15,1281816,53757814,10893687,0.2499,57357,43259
20760,209132321,49817093,49817093,16-FEB-15,1281816,53757804,11116570,0.2501,57359,43259
20798,209132321,49817093,49817093,16-FEB-15,1281816,53757818,11116570,0.2500,57353,43259
20831,209132321,49817093,49817093,16-FEB-15,1281816,53757814,11116570,0.2498,57357,43259
20832,209132321,49817093,49817093,16-FEB-15,1281816,53757806,11116570,0.2500,57358,43259
24274,216758914,51570665,0,03-APR-15,1281816,53757806,10495145,0.2500,57358,43259
24789,216758914,51570665,0,03-APR-15,1281816,53757814,10495145,0.2498,57357,43259


While implicit recommendations excel where data is sparse, it can often be helpful to make the interactions matrix a little more dense. We limited our data collection to models that had at least 5 likes. However, it may not be the case that every user has liked at least 5 models. Let's go ahead and knock out users that liked fewer than 5 models. This could possibly mean that some models end up with fewer than 5 likes once these users are knocked out, so we will have to alternate back and forth knocking users and models out until things stabilize.

In [97]:
def threshold_likes(df, uid_min, mid_min):
    n_users = df.uid.unique().shape[0]
    n_items = df.mid.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        mid_counts = df.groupby('uid').mid.count()
        df = df[~df.uid.isin(mid_counts[mid_counts < mid_min].index.tolist())]
        uid_counts = df.groupby('mid').uid.count()
        df = df[~df.mid.isin(uid_counts[uid_counts < uid_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby('uid').mid.count().min() >= mid_min)
    assert(df.groupby('mid').uid.count().min() >= uid_min)
    
    n_users = df.uid.unique().shape[0]
    n_items = df.mid.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

In [98]:
df_lim = threshold_likes(sample_renamed_df, 5, 5)

Starting likes info
Number of users: 904417
Number of models: 20820
Sparsity: 0.053%
Ending likes info
Number of users: 390967
Number of models: 19048
Sparsity: 0.122%


Nice, we're above 0.1% which should be suitable for making decent recommendations. We now need to map each uid and mid to a row and column, respectively, for our interactions, or "likes" matrix. This can be done simply with Python dictionaries

In [99]:
# Create mappings
mid_to_idx = {}
idx_to_mid = {}
for (idx, mid) in enumerate(df_lim.mid.unique().tolist()):
    mid_to_idx[mid] = idx
    idx_to_mid[idx] = mid
    
uid_to_idx = {}
idx_to_uid = {}
for (idx, uid) in enumerate(df_lim.uid.unique().tolist()):
    uid_to_idx[uid] = idx
    idx_to_uid[idx] = uid

In [100]:
mid_to_idx

{1266101: 0,
 1641805: 1,
 1598726: 2,
 1700807: 3,
 1474715: 4,
 1347705: 5,
 858448: 6,
 1129516: 7,
 1289573: 8,
 2062183: 9,
 1710006: 10,
 1496808: 11,
 1989919: 12,
 871080: 13,
 1886303: 14,
 1890170: 15,
 1780170: 16,
 1792181: 17,
 1160614: 18,
 579289: 19,
 1743269: 20,
 1950132: 21,
 417195: 22,
 2317970: 23,
 1754088: 24,
 1248121: 25,
 1679153: 26,
 1594063: 27,
 1362383: 28,
 2045979: 29,
 2002709: 30,
 1492070: 31,
 1636746: 32,
 2158086: 33,
 1159722: 34,
 15401: 35,
 3663: 36,
 1816861: 37,
 2046431: 38,
 1910476: 39,
 2035543: 40,
 1592529: 41,
 1612212: 42,
 629161: 43,
 1924519: 44,
 1438253: 45,
 2300953: 46,
 1506561: 47,
 1610645: 48,
 1994058: 49,
 1396356: 50,
 854094: 51,
 1487089: 52,
 859614: 53,
 1868246: 54,
 874053: 55,
 1977545: 56,
 1816431: 57,
 20320: 58,
 21430: 59,
 1160211: 60,
 1323999: 61,
 1569124: 62,
 1700668: 63,
 1916819: 64,
 1607423: 65,
 1860764: 66,
 1481653: 67,
 2326804: 68,
 1447915: 69,
 1732110: 70,
 2033462: 71,
 1269809: 72,
 2710

The last step is to actually build the matrix. We will use sparse matrices so as not to take up too much memory. Sparse matrices are tricky because they come in many forms, and there are giant performance tradeoffs between them. Below is a super slow way to build a likes matrix. I tried running %%timeit but got bored waiting for it to finish.

In [101]:
def map_ids(row, mapper):
    return mapper[row]

In [102]:
%%timeit
I = df_lim.uid.apply(map_ids, args=[uid_to_idx]).as_matrix()
J = df_lim.mid.apply(map_ids, args=[mid_to_idx]).as_matrix()
V = np.ones(I.shape[0])
likes = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
likes = likes.tocsr()

15.3 s ± 1.09 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [103]:
I = df_lim.uid.apply(map_ids, args=[uid_to_idx]).as_matrix()
J = df_lim.mid.apply(map_ids, args=[mid_to_idx]).as_matrix()
V = np.ones(I.shape[0])
likes = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
likes = likes.tocsr()

## Cross-validation: Splitsville

Okay, we got a likes matrix and need to split it into training and test matrices. I do this a bit trickily (which is maybe a word?). I would like to track precision@k as my optimization metric later. A k of 5 would be nice. However, if I move 5 items from training to test for some of the users, then they may not have any data left in the training set (remember they had a minimum 5 likes). Thus, the train_test_split only looks for people who have at least 2*k (10 in this case) likes before moving some of their data to the test set. This obviously biases our cross-validation towards users with more likes. So it goe

In [104]:
def train_test_split(ratings, split_count, fraction=None):
    """
    Split recommendation data into train and test sets
    
    Params
    ------
    ratings : scipy.sparse matrix
        Interactions between users and items.
    split_count : int
        Number of user-item-interactions per user to move
        from training to test set.
    fractions : float
        Fraction of users to split off some of their
        interactions into test set. If None, then all 
        users are considered.
    """
    # Note: likely not the fastest way to do things below.
    train = ratings.copy().tocoo()
    test = sparse.lil_matrix(train.shape)
    
    if fraction:
        try:
            user_index = np.random.choice(
                np.where(np.bincount(train.row) >= split_count * 2)[0], 
                replace=False,
                size=np.int32(np.floor(fraction * train.shape[0]))
            ).tolist()
        except:
            print(('Not enough users with > {} '
                  'interactions for fraction of {}')\
                  .format(2*k, fraction))
            raise
    else:
        user_index = range(train.shape[0])
        
    train = train.tolil()

    for user in user_index:
        test_ratings = np.random.choice(ratings.getrow(user).indices, 
                                        size=split_count, 
                                        replace=False)
        train[user, test_ratings] = 0.
        # These are just 1.0 right now
        test[user, test_ratings] = ratings[user, test_ratings]
   
    
    # Test and training are truly disjoint
    assert(train.multiply(test).nnz == 0)
    return train.tocsr(), test.tocsr(), user_index

In [105]:
train, test, user_index = train_test_split(likes, 5, fraction=0.2)

## Cross-validation: Grid search

Now that the data is split into training and test matrices, let's run a giant grid search to optimize our hyperparameters. We have four parameters that we would like to optimize:
1. num_factors: The number of latent factors, or degree of dimensionality in our model.
2. regularization: Scale of regularization for both user and item factors.
3. alpha: Our confidence scaling term.
4. iterations: Number of iterations to run Alternating Least Squares optimization.

I am going to track mean squared error (MSE) and precision at k (p@k), but I mainly care about the later. I've written some functions below to help with metric calculations and making the training log printout look nice. I am going to calculate a bunch of learning curves (that is, evaluate performance metrics at various stages of the training process) for a bunch of different hyperparameter combinations. Props to scikit-learn for being open source and letting me basically crib their GridSearchCV code

In [106]:
from sklearn.metrics import mean_squared_error
def calculate_mse(model, ratings, user_index=None):
    preds = model.predict_for_customers()
    if user_index:
        return mean_squared_error(ratings[user_index, :].toarray().ravel(),
                                  preds[user_index, :].ravel())
    
    return mean_squared_error(ratings.toarray().ravel(),
                              preds.ravel())

In [107]:
def precision_at_k(model, ratings, k=5, user_index=None):
    if not user_index:
        user_index = range(ratings.shape[0])
    ratings = ratings.tocsr()
    precisions = []
    # Note: line below may become infeasible for large datasets.
    predictions = model.predict_for_customers()
    for user in user_index:
        # In case of large dataset, compute predictions row-by-row like below
        # predictions = np.array([model.predict(row, i) for i in xrange(ratings.shape[1])])
        top_k = np.argsort(-predictions[user, :])[:k]
        labels = ratings.getrow(user).indices
        precision = float(len(set(top_k) & set(labels))) / float(k)
        precisions.append(precision)
    return np.mean(precisions)        

In [108]:
def print_log(row, header=False, spacing=12):
    top = ''
    middle = ''
    bottom = ''
    for r in row:
        top += '+{}'.format('-'*spacing)
        if isinstance(r, str):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, int):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, float):
            middle += '| {0:^{1}.5f} '.format(r, spacing-2)
        bottom += '+{}'.format('='*spacing)
    top += '+'
    middle += '|'
    bottom += '+'
    if header:
        print(top)
        print(middle)
        print(bottom)
    else:
        print(middle)
        print(top)

In [109]:
def learning_curve(model, train, test, epochs, k=5, user_index=None):
    if not user_index:
        user_index = range(train.shape[0])
    prev_epoch = 0
    train_precision = []
    train_mse = []
    test_precision = []
    test_mse = []
    
    headers = ['epochs', 'p@k train', 'p@k test',
               'mse train', 'mse test']
    print_log(headers, header=True)
    
    for epoch in epochs:
        model.iterations = epoch - prev_epoch
        if not hasattr(model, 'user_vectors'):
            model.fit(train)
        else:
            model.fit_partial(train)
        train_mse.append(calculate_mse(model, train, user_index))
        train_precision.append(precision_at_k(model, train, k, user_index))
        test_mse.append(calculate_mse(model, test, user_index))
        test_precision.append(precision_at_k(model, test, k, user_index))
        row = [epoch, train_precision[-1], test_precision[-1],
               train_mse[-1], test_mse[-1]]
        print_log(row)
        prev_epoch = epoch
    return model, train_precision, train_mse, test_precision, test_mse

In [110]:
def grid_search_learning_curve(base_model, train, test, param_grid,
                               user_index=None, patk=5, epochs=range(2, 40, 2)):
    """
    "Inspired" (stolen) from sklearn gridsearch
    https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py
    """
    curves = []
    keys, values = zip(*param_grid.items())
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        this_model = copy.deepcopy(base_model)
        print_line = []
        for k, v in params.items():
            setattr(this_model, k, v)
            print_line.append((k, v))

        print(' | '.join('{}: {}'.format(k, v) for (k, v) in print_line))
        _, train_patk, train_mse, test_patk, test_mse = learning_curve(this_model, train, test,
                                                                epochs, k=patk, user_index=user_index)
        curves.append({'params': params,
                       'patk': {'train': train_patk, 'test': test_patk},
                       'mse': {'train': train_mse, 'test': test_mse}})
    return curves

Please note that the below parameter grid is fucking huge and took like 2 days to run on my 6-year old 4-core i5. It turns out that the performance metrics functions are actually a good bit slower than the training process. These functions could be simply paralellized - something for me to do on a later date.

In [111]:
# param_grid = {'num_factors': [10, 20, 40, 80, 120],
#              'regularization': [0.0, 1e-5, 1e-3, 1e-1, 1e1, 1e2],
#              'alpha': [1, 10, 50, 100, 500, 1000]}
param_grid = {'num_factors': [10, 20],
              'regularization': [0.0, 1e-5],
              'alpha': [1 ]}

In [112]:
#base_model = implicit.als.AlternatingLeastSquares(factors=10)

In [113]:
#curves = grid_search_learning_curve(base_model, train, test,
#                                    param_grid,
#                                    user_index=user_index,
#                                    patk=5)

# fixing implicit implementation:

In [114]:
base_model = implicit.als.AlternatingLeastSquares()

In [115]:
# train the model on a sparse matrix of item/user/confidence weights
base_model.fit(likes)

100%|██████████| 15.0/15 [00:48<00:00,  2.95s/it]


In [116]:
user_items = likes.T.tocsr()

In [117]:
recommendations = base_model.recommend(3334225, user_items)

# find related items
related = base_model.similar_items(67244)

IndexError: index 3334225 is out of bounds for axis 0 with size 19048

In [78]:
recommendations

[(4973, 0.4024711),
 (2824, 0.37532645),
 (46948, 0.33600608),
 (73047, 0.31712249),
 (2418, 0.30940416),
 (1792, 0.29819414),
 (18077, 0.29223019),
 (33470, 0.27846566),
 (10245, 0.27767515),
 (15866, 0.27399385)]

In [79]:
related

[(67244, 0.99999988),
 (343980, 0.70765382),
 (209950, 0.69686389),
 (345297, 0.69174719),
 (115368, 0.69076794),
 (348882, 0.68712658),
 (280172, 0.68344337),
 (312709, 0.67982864),
 (213122, 0.67966419),
 (47662, 0.67385834)]